In [1]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
from scipy.stats import norm
import scipy.optimize as spo
import numpy as np

AAPL_call = pd.read_excel('Apple.xlsx','Call',header=0,converters={'Price':float,'Strike':float})
print("Column headings:")
print(AAPL_call.columns)
AAPL_call = AAPL_call[['Price','Strike']]
print(AAPL_call['Strike'])
print(AAPL_call['Price'])

S = 315.24
r = 0.0145 # 6m libor since the dataset cames from USA
TTM = 1 #simplified version: maturity is fixed



Column headings:
Index(['Price', 'Strike', 'Maturity'], dtype='object')
0     140.0
1     145.0
2     150.0
3     160.0
4     165.0
5     170.0
6     175.0
7     180.0
8     185.0
9     190.0
10    200.0
11    205.0
12    210.0
13    220.0
14    225.0
15    230.0
16    240.0
17    250.0
18    260.0
19    270.0
20    280.0
21    290.0
22    300.0
23    310.0
24    320.0
25    330.0
26    340.0
27    350.0
28    370.0
Name: Strike, dtype: float64
0     174.85
1     170.25
2     163.31
3     156.05
4     151.92
5     145.95
6     140.17
7     135.40
8     132.25
9     127.60
10    118.00
11    113.35
12    109.30
13     99.35
14     94.76
15     91.92
16     83.03
17     75.43
18     66.50
19     61.17
20     54.35
21     47.88
22     42.07
23     36.80
24     31.85
25     27.45
26     23.50
27     19.85
28     14.30
Name: Price, dtype: float64


In [4]:
def bls(S,K,Vol,r,TTM):
    d1 = (np.log(S/K) + (r + 0.5 * Vol**2)*TTM)/(np.sqrt(TTM)*Vol)
    d2 = d1 - Vol * np.sqrt(TTM)
    C = S * norm.cdf(d1) - K * norm.cdf(d2) * np.exp(- r * TTM)
    return C

In [5]:
def diff_bls(Vol):
    return sum((AAPL_call['Price'] - bls(S, AAPL_call['Strike'],Vol,r,TTM)) ** 2)/2

def diff_bls_vec(Vol):
    return ((AAPL_call['Price'] - bls(S, AAPL_call['Strike'],Vol,r,TTM)))


In [6]:
len(AAPL_call['Price'])

29

In [7]:
from scipy.optimize import least_squares
Vol1 = least_squares(diff_bls_vec ,1, bounds=(0, np.inf))
print(Vol1)
print(diff_bls(0.2))
print(diff_bls(Vol1.x))
# print(data['Price'] - bls(price, data['Strike'],Vol1['x'],0.01,data['Expiry']))
# data['Price'] - bls(price, data['Strike'],Vol,0.01,data['Expiry'])

 active_mask: array([0])
        cost: 29.518920178858412
         fun: array([-2.41298888, -2.09031899, -4.11065966, -1.54592577, -0.77463702,
       -1.85403262, -2.75722894, -2.66788729, -0.98022208, -0.81899182,
       -0.89741395, -0.84897947, -0.25067234, -1.08163115, -1.21479442,
        0.32432085, -0.06757109,  0.44167482, -0.81251487,  1.063808  ,
        0.95443704,  0.68356902,  0.55566655,  0.45475243,  0.17328898,
       -0.03920144, -0.25763297, -0.60285211, -0.69499095])
        grad: array([-0.00454712])
         jac: array([[  -0.40070534],
       [  -0.63507462],
       [  -0.97300339],
       [  -2.08601379],
       [  -2.93286324],
       [  -4.02451897],
       [  -5.39969826],
       [  -7.09529686],
       [  -9.1445179 ],
       [ -11.57509804],
       [ -17.65445518],
       [ -21.31812668],
       [ -25.39129066],
       [ -34.68501091],
       [ -39.84029388],
       [ -45.27607346],
       [ -56.769701  ],
       [ -68.67759514],
       [ -80.4688797 ],
   

In [9]:
Volat = abs(np.random.normal(Vol1.x,0.05,10000))
dataset_ML = np.array(AAPL_call['Price'])
dataset_ML = np.append(dataset_ML,Vol1.x)
Call_sim = []
for i in range(0, len(Volat)):
    Call_sim = bls(S,AAPL_call['Strike'],Volat[i],r,TTM)
    Call_sim = np.append(Call_sim,Volat[i])
    dataset_ML = np.vstack((dataset_ML,Call_sim))



In [10]:
dataset_ML # row: prices for different strikes + volat (last column)

array([[174.85      , 170.25      , 163.31      , ...,  19.85      ,
         14.3       ,   0.25309932],
       [177.2590202 , 172.33389091, 167.41060944, ...,  18.86175368,
         13.52624358,   0.24007087],
       [177.25838106, 172.33281992, 167.40888181, ...,  18.48264638,
         13.17885357,   0.23695397],
       ...,
       [177.25641511, 172.32940234, 167.40317624, ...,  16.62976981,
         11.49713744,   0.22163785],
       [177.25625378, 172.32910815, 167.40266305, ...,  16.36834522,
         11.26221463,   0.21946459],
       [177.25537971, 172.32739747, 167.39946797, ...,  12.11846675,
          7.54726076,   0.18355522]])

In [11]:
df = pd.DataFrame(dataset_ML)
print(np.size(df,1)) # 29 strikes + Volat
df[df.columns[-1]]

30


0        0.253099
1        0.240071
2        0.236954
3        0.215291
4        0.242520
           ...   
9996     0.301931
9997     0.272355
9998     0.221638
9999     0.219465
10000    0.183555
Name: 29, Length: 10001, dtype: float64

In [12]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


x = df[df.columns[0:-1]]
print(x)
y = df[df.columns[-1]]
print(y)
x_train, x_test, y_train, y_test = train_test_split(x, y , train_size = 0.7, random_state =  42)
x_test_fin = x_test

               0           1           2           3           4           5   \
0      174.850000  170.250000  163.310000  156.050000  151.920000  145.950000   
1      177.259020  172.333891  167.410609  157.573577  152.662694  147.759522   
2      177.258381  172.332820  167.408882  157.569517  152.656747  147.751045   
3      177.256001  172.328640  167.401833  157.551557  152.629455  147.710782   
4      177.259596  172.334845  167.412133  157.577091  152.667798  147.766740   
...           ...         ...         ...         ...         ...         ...   
9996   177.314022  172.414464  167.525593  157.791731  152.953712  148.140292   
9997   177.274473  172.357899  167.446749  157.648800  152.767157  147.901300   
9998   177.256415  172.329402  167.403176  157.555249  152.635255  147.719601   
9999   177.256254  172.329108  167.402663  157.553862  152.633093  147.716339   
10000  177.255380  172.327397  167.399468  157.544037  152.616819  147.690290   

               6           

In [13]:
# Normalize Training Data 
std_scale = preprocessing.StandardScaler().fit(x_train) #Standardize features by removing the mean and scaling to unit variance
x_train_norm = std_scale.transform(x_train)

#Converting numpy array to dataframe
training_norm_col = pd.DataFrame(x_train_norm, index=x_train.index, columns=x_train.columns) 
x_train.update(training_norm_col)


# Normalize Testing Data by using mean and SD of training set
x_test_norm = std_scale.transform(x_test)
testing_norm_col = pd.DataFrame(x_test, index=x_test.index, columns=x_test.columns) 
x_test.update(testing_norm_col)


C:\Users\Daniele\Anaconda3\lib\site-packages\pandas\core\frame.py:5804: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)
C:\Users\Daniele\Anaconda3\lib\site-packages\pandas\core\frame.py:5804: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


In [14]:
import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
import timeit
x_train_norm=np.asarray(x_train_norm)
y_train=np.asarray(y_train)
x_test_norm= np.asarray(x_test_norm)
y_test= np.asarray(y_test)

X = x_train_norm
y = y_train

# define the keras model
model_mae = keras.models.Sequential()
model_mae.add(keras.layers.Dense(29, input_dim=29, activation='tanh')) #29 is the input size
model_mae.add(keras.layers.Dense(64, activation='relu'))
model_mae.add(keras.layers.Dense(64, activation='tanh'))
model_mae.add(keras.layers.Dense(64, activation='relu'))
model_mae.add(keras.layers.Dense(1))

model_mae.compile(optimizer='adam',
              loss='mae',
              metrics=['mae'])
# compile the keras model
# fit the keras model on the dataset
history_mae = model_mae.fit(X, y, epochs= 1200, batch_size = 500, verbose = 1)
# evaluate the keras model
#"""
# elapsed_time = timeit.timeit(code_to_test, number=100)/100
# print(elapsed_time)

Train on 7000 samples
Epoch 1/1200
7000/7000 [==============================] - 1s 98us/sample - loss: 0.0643 - mae: 0.0643
Epoch 2/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 0.0189 - mae: 0.0189
Epoch 3/1200
7000/7000 [==============================] - 0s 9us/sample - loss: 0.0076 - mae: 0.0076
Epoch 4/1200
7000/7000 [==============================] - 0s 7us/sample - loss: 0.0069 - mae: 0.0069
Epoch 5/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 0.0068 - mae: 0.0068
Epoch 6/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 0.0092 - mae: 0.0092
Epoch 7/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 0.0070 - mae: 0.0070
Epoch 8/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 0.0088 - mae: 0.0088
Epoch 9/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 0.0065 - mae: 0.0065
Epoch 10/1200
7000/7000 [==============================] - 0s 

7000/7000 [==============================] - 0s 10us/sample - loss: 0.0022 - mae: 0.00220s - loss: 0.0022 - mae: 0.002
Epoch 80/1200
7000/7000 [==============================] - 0s 15us/sample - loss: 0.0029 - mae: 0.0029
Epoch 81/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 0.0025 - mae: 0.0025
Epoch 82/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 0.0023 - mae: 0.0023
Epoch 83/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 0.0021 - mae: 0.0021
Epoch 84/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 0.0020 - mae: 0.0020
Epoch 85/1200
7000/7000 [==============================] - 0s 9us/sample - loss: 0.0038 - mae: 0.0038
Epoch 86/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 0.0037 - mae: 0.0037
Epoch 87/1200
7000/7000 [==============================] - 0s 8us/sample - loss: 0.0038 - mae: 0.0038
Epoch 88/1200
7000/7000 [==============================] - 

7000/7000 [==============================] - 0s 12us/sample - loss: 0.0023 - mae: 0.0023
Epoch 158/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 0.0023 - mae: 0.0023
Epoch 159/1200
7000/7000 [==============================] - 0s 15us/sample - loss: 0.0019 - mae: 0.0019
Epoch 160/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 0.0023 - mae: 0.0023
Epoch 161/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 0.0019 - mae: 0.0019
Epoch 162/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 0.0022 - mae: 0.0022
Epoch 163/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 0.0024 - mae: 0.0024
Epoch 164/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 0.0022 - mae: 0.0022
Epoch 165/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 0.0020 - mae: 0.0020
Epoch 166/1200
7000/7000 [==============================] - 0s 10us/sample - lo

7000/7000 [==============================] - 0s 19us/sample - loss: 0.0016 - mae: 0.0016
Epoch 235/1200
7000/7000 [==============================] - 0s 17us/sample - loss: 0.0016 - mae: 0.0016
Epoch 236/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 0.0022 - mae: 0.0022
Epoch 237/1200
7000/7000 [==============================] - 0s 15us/sample - loss: 0.0025 - mae: 0.0025
Epoch 238/1200
7000/7000 [==============================] - 0s 15us/sample - loss: 0.0024 - mae: 0.0024
Epoch 239/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 0.0019 - mae: 0.0019
Epoch 240/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 0.0014 - mae: 0.0014
Epoch 241/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 0.0011 - mae: 0.0011
Epoch 242/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 0.0012 - mae: 0.0012
Epoch 243/1200
7000/7000 [==============================] - 0s 14us/sample - lo

7000/7000 [==============================] - 0s 14us/sample - loss: 0.0014 - mae: 0.0014
Epoch 312/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 0.0011 - mae: 0.0011
Epoch 313/1200
7000/7000 [==============================] - 0s 15us/sample - loss: 0.0010 - mae: 0.0010 
Epoch 314/1200
7000/7000 [==============================] - 0s 20us/sample - loss: 0.0013 - mae: 0.0013
Epoch 315/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 0.0020 - mae: 0.0020
Epoch 316/1200
7000/7000 [==============================] - 0s 9us/sample - loss: 0.0022 - mae: 0.0022
Epoch 317/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 0.0017 - mae: 0.0017
Epoch 318/1200
7000/7000 [==============================] - 0s 20us/sample - loss: 0.0014 - mae: 0.0014
Epoch 319/1200
7000/7000 [==============================] - 0s 22us/sample - loss: 0.0012 - mae: 0.0012
Epoch 320/1200
7000/7000 [==============================] - 0s 13us/sample - lo

...

7000/7000 [==============================] - 0s 12us/sample - loss: 7.3419e-04 - mae: 7.3419e-04
Epoch 843/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 6.7138e-04 - mae: 6.7138e-04
Epoch 844/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 7.9636e-04 - mae: 7.9636e-04
Epoch 845/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 6.8480e-04 - mae: 6.8480e-04
Epoch 846/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 9.1006e-04 - mae: 9.1006e-04
Epoch 847/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 8.9834e-04 - mae: 8.9834e-04
Epoch 848/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 0.0014 - mae: 0.0014
Epoch 849/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 0.0010 - mae: 0.0010
Epoch 850/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 6.6164e-04 - mae: 6.6164e-04
Epoch 851/1200
7000/700

7000/7000 [==============================] - 0s 14us/sample - loss: 5.4738e-04 - mae: 5.4738e-04
Epoch 917/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 8.4320e-04 - mae: 8.4320e-04
Epoch 918/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 7.1995e-04 - mae: 7.1995e-04
Epoch 919/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 6.8600e-04 - mae: 6.8600e-04
Epoch 920/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 7.2946e-04 - mae: 7.2946e-04
Epoch 921/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 0.0010 - mae: 0.0010
Epoch 922/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 0.0012 - mae: 0.0012
Epoch 923/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 0.0013 - mae: 0.0013
Epoch 924/1200
7000/7000 [==============================] - 0s 15us/sample - loss: 0.0012 - mae: 0.0012
Epoch 925/1200
7000/7000 [=============

7000/7000 [==============================] - 0s 12us/sample - loss: 8.8978e-04 - mae: 8.8978e-04
Epoch 991/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 8.5579e-04 - mae: 8.5579e-04
Epoch 992/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 5.1521e-04 - mae: 5.1521e-04
Epoch 993/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 5.7212e-04 - mae: 5.7212e-04
Epoch 994/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 5.1610e-04 - mae: 5.1610e-04
Epoch 995/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 8.0501e-04 - mae: 8.0501e-04
Epoch 996/1200
7000/7000 [==============================] - 0s 15us/sample - loss: 9.3251e-04 - mae: 9.3251e-04
Epoch 997/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 9.2087e-04 - mae: 9.2087e-04
Epoch 998/1200
7000/7000 [==============================] - 0s 20us/sample - loss: 0.0011 - mae: 0.0011 
Epoch 999/1200

7000/7000 [==============================] - 0s 25us/sample - loss: 9.3463e-04 - mae: 9.3463e-04
Epoch 1064/1200
7000/7000 [==============================] - 0s 20us/sample - loss: 6.9579e-04 - mae: 6.9579e-04
Epoch 1065/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 8.5506e-04 - mae: 8.5506e-04
Epoch 1066/1200
7000/7000 [==============================] - 0s 21us/sample - loss: 8.1184e-04 - mae: 8.1184e-04
Epoch 1067/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 6.3322e-04 - mae: 6.3322e-04
Epoch 1068/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 6.2001e-04 - mae: 6.2001e-04
Epoch 1069/1200
7000/7000 [==============================] - 0s 19us/sample - loss: 0.0012 - mae: 0.0012
Epoch 1070/1200
7000/7000 [==============================] - 0s 21us/sample - loss: 9.9169e-04 - mae: 9.9169e-04
Epoch 1071/1200
7000/7000 [==============================] - 0s 22us/sample - loss: 0.0010 - mae: 0.0010
Epoch 1072/1200

Epoch 1136/1200
7000/7000 [==============================] - 0s 20us/sample - loss: 6.9970e-04 - mae: 6.9970e-04
Epoch 1137/1200
7000/7000 [==============================] - 0s 19us/sample - loss: 7.1341e-04 - mae: 7.1341e-04
Epoch 1138/1200
7000/7000 [==============================] - 0s 20us/sample - loss: 5.5201e-04 - mae: 5.5201e-04
Epoch 1139/1200
7000/7000 [==============================] - 0s 17us/sample - loss: 5.5674e-04 - mae: 5.5674e-04
Epoch 1140/1200
7000/7000 [==============================] - 0s 20us/sample - loss: 5.1415e-04 - mae: 5.1415e-04
Epoch 1141/1200
7000/7000 [==============================] - 0s 22us/sample - loss: 5.2840e-04 - mae: 5.2840e-04
Epoch 1142/1200
7000/7000 [==============================] - 0s 17us/sample - loss: 7.2131e-04 - mae: 7.2131e-04
Epoch 1143/1200
7000/7000 [==============================] - 0s 21us/sample - loss: 5.5819e-04 - mae: 5.5819e-04
Epoch 1144/1200
7000/7000 [==============================] - 0s 17us/sample - loss: 6.2563e-04 -

In [15]:
X = x_train_norm
y = y_train
# define the keras model
model_mse = keras.models.Sequential()
model_mse.add(keras.layers.Dense(29, input_dim=29, activation='tanh'))
model_mse.add(keras.layers.Dense(64, activation='relu'))
model_mse.add(keras.layers.Dense(64, activation='tanh'))
model_mse.add(keras.layers.Dense(64, activation='relu'))
model_mse.add(keras.layers.Dense(1))

model_mse.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])
# compile the keras model
# fit the keras model on the dataset
history1 = model_mse.fit(X, y, epochs= 80, batch_size = 500, verbose = 1)
# evaluate the keras model


Train on 7000 samples
Epoch 1/80
7000/7000 [==============================] - 1s 167us/sample - loss: 0.0349 - mae: 0.1400
Epoch 2/80
7000/7000 [==============================] - 0s 15us/sample - loss: 0.0033 - mae: 0.0428
Epoch 3/80
7000/7000 [==============================] - 0s 15us/sample - loss: 5.9414e-04 - mae: 0.0195
Epoch 4/80
7000/7000 [==============================] - 0s 10us/sample - loss: 1.2842e-04 - mae: 0.0088
Epoch 5/80
7000/7000 [==============================] - 0s 11us/sample - loss: 4.5002e-05 - mae: 0.0054
Epoch 6/80
7000/7000 [==============================] - 0s 12us/sample - loss: 2.1032e-05 - mae: 0.0036
Epoch 7/80
7000/7000 [==============================] - 0s 22us/sample - loss: 1.2881e-05 - mae: 0.0027
Epoch 8/80
7000/7000 [==============================] - 0s 12us/sample - loss: 9.0092e-06 - mae: 0.0023
Epoch 9/80
7000/7000 [==============================] - 0s 11us/sample - loss: 6.7657e-06 - mae: 0.0020
Epoch 10/80
7000/7000 [==========================

7000/7000 [==============================] - 0s 23us/sample - loss: 1.6322e-07 - mae: 3.0362e-04
Epoch 77/80
7000/7000 [==============================] - 0s 8us/sample - loss: 1.6138e-07 - mae: 3.0252e-04
Epoch 78/80
7000/7000 [==============================] - 0s 8us/sample - loss: 1.6018e-07 - mae: 3.0049e-04
Epoch 79/80
7000/7000 [==============================] - 0s 10us/sample - loss: 1.5955e-07 - mae: 2.9970e-04
Epoch 80/80
7000/7000 [==============================] - 0s 17us/sample - loss: 1.5287e-07 - mae: 2.9410e-04


In [16]:
X = x_train_norm
y = y_train
# define the keras model
model_lcl = keras.models.Sequential()
model_lcl.add(keras.layers.Dense(29, input_dim=29, activation='tanh'))
model_lcl.add(keras.layers.Dense(64, activation='relu'))
model_lcl.add(keras.layers.Dense(64, activation='tanh'))
model_lcl.add(keras.layers.Dense(64, activation='relu'))
model_lcl.add(keras.layers.Dense(1))

model_lcl.compile(optimizer='adam',
              loss='logcosh',
              metrics=['mae'])
# compile the keras model
# fit the keras model on the dataset

history2 = model_lcl.fit(X, y, epochs= 1200,batch_size = 500, verbose = 1)
# evaluate the keras model


Train on 7000 samples
Epoch 1/1200
7000/7000 [==============================] - 1s 133us/sample - loss: 0.0064 - mae: 0.0808
Epoch 2/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 6.6447e-04 - mae: 0.0281
Epoch 3/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 1.5750e-04 - mae: 0.0144
Epoch 4/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 4.0180e-05 - mae: 0.0072
Epoch 5/1200
7000/7000 [==============================] - 0s 19us/sample - loss: 1.2722e-05 - mae: 0.0041
Epoch 6/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 5.3412e-06 - mae: 0.0025
Epoch 7/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 3.2617e-06 - mae: 0.0020
Epoch 8/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 2.2132e-06 - mae: 0.0016
Epoch 9/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 1.7245e-06 - mae: 0.0014
Epoch 10/1200
7000/7000 [==

7000/7000 [==============================] - 0s 15us/sample - loss: 2.6382e-07 - mae: 4.6601e-04
Epoch 75/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 4.1566e-07 - mae: 5.7346e-04
Epoch 76/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 1.4740e-07 - mae: 3.6151e-04
Epoch 77/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 1.5561e-07 - mae: 3.7669e-04
Epoch 78/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 1.5396e-07 - mae: 3.7858e-04
Epoch 79/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 1.4294e-07 - mae: 3.5727e-04
Epoch 80/1200
7000/7000 [==============================] - 0s 15us/sample - loss: 8.5293e-08 - mae: 2.8083e-04
Epoch 81/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 9.3883e-08 - mae: 2.9948e-04
Epoch 82/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 1.9264e-07 - mae: 4.1330e-04
Epoch 83/1200
7

7000/7000 [==============================] - 0s 17us/sample - loss: 1.2743e-07 - mae: 3.4257e-04
Epoch 148/1200
7000/7000 [==============================] - 0s 26us/sample - loss: 7.9595e-08 - mae: 2.9018e-04
Epoch 149/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 8.1044e-08 - mae: 2.8626e-04
Epoch 150/1200
7000/7000 [==============================] - 0s 18us/sample - loss: 6.2773e-08 - mae: 2.5666e-04
Epoch 151/1200
7000/7000 [==============================] - 0s 30us/sample - loss: 6.5709e-08 - mae: 2.5781e-04
Epoch 152/1200
7000/7000 [==============================] - 0s 21us/sample - loss: 6.0806e-08 - mae: 2.4365e-04
Epoch 153/1200
7000/7000 [==============================] - 0s 17us/sample - loss: 4.8448e-08 - mae: 2.1606e-04
Epoch 154/1200
7000/7000 [==============================] - 0s 21us/sample - loss: 5.4242e-08 - mae: 2.2725e-04
Epoch 155/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 8.6725e-08 - mae: 2.8865e-04
Epoch 1

Epoch 220/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 2.3741e-07 - mae: 5.1476e-04
Epoch 221/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 5.9590e-08 - mae: 2.6058e-04
Epoch 222/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 1.7208e-07 - mae: 4.4965e-04
Epoch 223/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 7.2331e-08 - mae: 2.9004e-04
Epoch 224/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 1.8028e-07 - mae: 4.3882e-04
Epoch 225/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 3.2329e-06 - mae: 0.0020
Epoch 226/1200
7000/7000 [==============================] - 0s 12us/sample - loss: 4.9719e-06 - mae: 0.0025
Epoch 227/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 3.4628e-06 - mae: 0.0020
Epoch 228/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 4.4974e-06 - mae: 0.0024
Epoch 22

7000/7000 [==============================] - 0s 11us/sample - loss: 1.3927e-06 - mae: 0.0014
Epoch 294/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 6.1453e-07 - mae: 9.1981e-04
Epoch 295/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 8.8978e-07 - mae: 0.0010
Epoch 296/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 1.2662e-06 - mae: 0.0013
Epoch 297/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 2.2195e-06 - mae: 0.0015
Epoch 298/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 6.5331e-06 - mae: 0.0029
Epoch 299/1200
7000/7000 [==============================] - 0s 13us/sample - loss: 8.8511e-07 - mae: 0.0011
Epoch 300/1200
7000/7000 [==============================] - 0s 17us/sample - loss: 3.1077e-07 - mae: 6.3619e-04
Epoch 301/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 3.0491e-07 - mae: 6.1995e-04
Epoch 302/1200
7000/7000 [=====

...

7000/7000 [==============================] - 0s 9us/sample - loss: 1.2826e-07 - mae: 3.9780e-04
Epoch 882/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 6.2658e-07 - mae: 7.9426e-04
Epoch 883/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 3.4644e-06 - mae: 0.0022
Epoch 884/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 1.2449e-06 - mae: 0.0013
Epoch 885/1200
7000/7000 [==============================] - 0s 9us/sample - loss: 4.5633e-07 - mae: 7.4798e-04
Epoch 886/1200
7000/7000 [==============================] - 0s 11us/sample - loss: 2.4945e-07 - mae: 5.5714e-04
Epoch 887/1200
7000/7000 [==============================] - 0s 9us/sample - loss: 3.6578e-07 - mae: 6.8496e-04
Epoch 888/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 3.1113e-07 - mae: 6.0802e-04
Epoch 889/1200
7000/7000 [==============================] - 0s 10us/sample - loss: 2.0047e-07 - mae: 4.9229e-04
Epoch 890/1200
700

7000/7000 [==============================] - 0s 25us/sample - loss: 2.8921e-08 - mae: 1.5662e-04
Epoch 955/1200
7000/7000 [==============================] - 0s 18us/sample - loss: 2.5049e-08 - mae: 1.4492e-04
Epoch 956/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 2.4426e-08 - mae: 1.4136e-04
Epoch 957/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 2.3400e-08 - mae: 1.4055e-04
Epoch 958/1200
7000/7000 [==============================] - 0s 19us/sample - loss: 2.3631e-08 - mae: 1.3458e-04
Epoch 959/1200
7000/7000 [==============================] - 0s 33us/sample - loss: 2.4751e-08 - mae: 1.4052e-04
Epoch 960/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 2.3514e-08 - mae: 1.3655e-04
Epoch 961/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 2.2343e-08 - mae: 1.3156e-04
Epoch 962/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 2.2218e-08 - mae: 1.2968e-04
Epoch 9

7000/7000 [==============================] - 0s 17us/sample - loss: 2.7422e-08 - mae: 1.6458e-04
Epoch 1027/1200
7000/7000 [==============================] - 0s 19us/sample - loss: 2.0204e-08 - mae: 1.2713e-04
Epoch 1028/1200
7000/7000 [==============================] - 0s 20us/sample - loss: 9.2405e-08 - mae: 3.2298e-04
Epoch 1029/1200
7000/7000 [==============================] - 0s 18us/sample - loss: 9.4684e-07 - mae: 0.0011
Epoch 1030/1200
7000/7000 [==============================] - 0s 15us/sample - loss: 1.0200e-06 - mae: 0.0012
Epoch 1031/1200
7000/7000 [==============================] - 0s 18us/sample - loss: 5.2461e-07 - mae: 7.9541e-04
Epoch 1032/1200
7000/7000 [==============================] - 0s 25us/sample - loss: 2.9995e-07 - mae: 6.0763e-04
Epoch 1033/1200
7000/7000 [==============================] - 0s 24us/sample - loss: 7.6812e-08 - mae: 2.9512e-04
Epoch 1034/1200
7000/7000 [==============================] - 0s 22us/sample - loss: 4.4611e-08 - mae: 2.1865e-04
Epoch 1

7000/7000 [==============================] - 0s 15us/sample - loss: 4.1544e-08 - mae: 2.0097e-04
Epoch 1099/1200
7000/7000 [==============================] - 0s 19us/sample - loss: 2.0470e-08 - mae: 1.2965e-04
Epoch 1100/1200
7000/7000 [==============================] - 0s 17us/sample - loss: 9.8849e-07 - mae: 0.0011
Epoch 1101/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 2.6799e-06 - mae: 0.0020
Epoch 1102/1200
7000/7000 [==============================] - 0s 15us/sample - loss: 3.9654e-07 - mae: 7.1747e-04
Epoch 1103/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 1.7344e-07 - mae: 4.7785e-04
Epoch 1104/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 1.9887e-07 - mae: 5.0319e-04
Epoch 1105/1200
7000/7000 [==============================] - 0s 15us/sample - loss: 1.3061e-07 - mae: 4.0418e-04
Epoch 1106/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 1.1960e-07 - mae: 3.8902e-04
Epoch 1

7000/7000 [==============================] - 0s 17us/sample - loss: 4.6163e-08 - mae: 2.3099e-04
Epoch 1171/1200
7000/7000 [==============================] - 0s 15us/sample - loss: 3.2529e-07 - mae: 6.4402e-04
Epoch 1172/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 3.9312e-07 - mae: 7.0109e-04
Epoch 1173/1200
7000/7000 [==============================] - 0s 19us/sample - loss: 1.2527e-06 - mae: 0.0012
Epoch 1174/1200
7000/7000 [==============================] - 0s 19us/sample - loss: 1.1529e-06 - mae: 0.0012
Epoch 1175/1200
7000/7000 [==============================] - 0s 19us/sample - loss: 3.8044e-07 - mae: 6.7310e-04
Epoch 1176/1200
7000/7000 [==============================] - 0s 14us/sample - loss: 8.5324e-07 - mae: 0.0011
Epoch 1177/1200
7000/7000 [==============================] - 0s 17us/sample - loss: 4.2274e-07 - mae: 7.5242e-04
Epoch 1178/1200
7000/7000 [==============================] - 0s 16us/sample - loss: 2.5501e-07 - mae: 5.6012e-04
Epoch 1179/

In [17]:
print('Mean Absolute Error')
pred_train=model_mae.predict(x_train_norm)
pred_test=model_mae.predict(x_test_norm)
print(np.sum((np.ravel(pred_train)-y_train)**2)/pred_train.shape[0])
print(np.sum((np.ravel(pred_test)-y_test)**2)/pred_test.shape[0])

print('Mean Square Error')
pred_train=model_mse.predict(x_train_norm)
pred_test=model_mse.predict(x_test_norm)
print(np.sum((np.ravel(pred_train)-y_train)**2)/pred_train.shape[0])
print(np.sum((np.ravel(pred_test)-y_test)**2)/pred_test.shape[0])

print('logcosh')
pred_train=model_lcl.predict(x_train_norm)
pred_test=model_lcl.predict(x_test_norm)
print(np.sum((np.ravel(pred_train)-y_train)**2)/pred_train.shape[0])
print(np.sum((np.ravel(pred_test)-y_test)**2)/pred_test.shape[0])



Mean Absolute Error
9.549838213581152e-07
1.1056823358687077e-06
Mean Square Error
1.5045173468268422e-07
2.7863272217741588e-06
logcosh
2.711646756993096e-07
8.971397668907609e-06


In [15]:
# Now Input: [Strikes] -> Output: [Sigma]
# Extend: perturbate S_0, rf, T, Input [S_0, rf, T, Strikes] -> Output: [Sigma]